# [370. 区间加法](https://leetcode.cn/problems/range-addition)

[English Version](/solution/0300-0399/0370.Range%20Addition/README_EN.md)

## 题目描述

<!-- 这里写题目描述 -->

<p>假设你有一个长度为&nbsp;<em><strong>n</strong></em>&nbsp;的数组，初始情况下所有的数字均为&nbsp;<strong>0</strong>，你将会被给出&nbsp;<em><strong>k</strong></em>​​​​​​<em>​</em> 个更新的操作。</p>

<p>其中，每个操作会被表示为一个三元组：<strong>[startIndex, endIndex, inc]</strong>，你需要将子数组&nbsp;<strong>A[startIndex ... endIndex]</strong>（包括 startIndex 和 endIndex）增加&nbsp;<strong>inc</strong>。</p>

<p>请你返回&nbsp;<strong><em>k</em></strong>&nbsp;次操作后的数组。</p>

<p><strong>示例:</strong></p>

<pre><strong>输入: </strong>length = 5, updates = [[1,3,2],[2,4,3],[0,2,-2]]
<strong>输出: </strong>[-2,0,3,5,3]
</pre>

<p><strong>解释:</strong></p>

<pre>初始状态:
[0,0,0,0,0]

进行了操作 [1,3,2] 后的状态:
[0,2,2,2,0]

进行了操作 [2,4,3] 后的状态:
[0,2,5,5,3]

进行了操作 [0,2,-2] 后的状态:
[-2,0,3,5,3]
</pre>

## 解法

<!-- 这里可写通用的实现逻辑 -->

**方法一：前缀和**

设 d 为前缀和。

给区间 [l, r] 中的每一个数加上 c，即 `d[l] += c, d[r + 1] -= c`。

对 d 求前缀和，即可得到操作后的数组。

时间复杂度 O(n)。

**方法二：树状数组 + 差分思想**

时间复杂度 O(nlogn)。

树状数组，也称作“二叉索引树”（Binary Indexed Tree）或 Fenwick 树。 它可以高效地实现如下两个操作：

1. **单点更新** `update(x, delta)`： 把序列 x 位置的数加上一个值 delta；
1. **前缀和查询** `query(x)`：查询序列 `[1,...x]` 区间的区间和，即位置 x 的前缀和。

这两个操作的时间复杂度均为 `O(log n)`。

注意，其实差分数组更快，这提示我们有时候，区域加减法不一定都要树状数组，单纯的前缀和思想就像


In [7]:
from typing import List
class Solution:
    def getModifiedArray(self, length: int, updates: List[List[int]]) -> List[int]:
        #差分数组 差分数组其实更快
        dif_arr=[0]*(length+1)
        for start,end,value in updates:#这里注意，本身可以将list切片！
            dif_arr[start]=value
            dif_arr[end+1]=-value# 就是一起一落
        s=[0]*(length+1+1)
        print(dif_arr)
        for i in range(length):
            s[i+1]=s[i]+dif_arr[i]
        return s[1:-1]
        
if __name__=="__main__":
    solution=Solution()
    print(solution.getModifiedArray(5,[[1,3,2]]))
    
    print(solution.getModifiedArray(5,[[1,3,2],[2,4,3],[0,2,-2]]))

[0, 2, 0, 0, -2, 0]
[0, 2, 2, 2, 0]
[-2, 2, 3, 2, -2, -3]
[-2, 0, 3, 5, 3]


In [ ]:
10 
01+01
10


<!-- tabs:start -->

### **Python3**

<!-- 这里可写当前语言的特殊实现逻辑 -->

差分数组：

```python


class Solution:
    def getModifiedArray(self, length: int, updates: List[List[int]]) -> List[int]:
        delta = [0] * length
        for start, end, inc in updates:
            delta[start] += inc
            if end + 1 < length:
                delta[end + 1] -= inc
        return list(accumulate(delta))
```


In [6]:
from typing import List

class BinaryIndexedTree:
    def __init__(self, n):
        self.n = n
        self.c = [0] * (n + 1)

    @staticmethod
    def lowbit(x):#取得最后一位的一（子节点+lowbit==父节点 父节点-lowbit==子节点）
        return x & -x

    def update(self, x, delta):#将BIT覆盖的区间包含A[i]的值加上delta
        while x <= self.n:
            self.c[x] += delta
            x += BinaryIndexedTree.lowbit(x)#不需要类和实例化直接调用即可,更快

    def query(self, x):#求前缀和
        s = 0
        while x:
            s += self.c[x]
            x -= BinaryIndexedTree.lowbit(x)#
        return s


class Solution:
    def getModifiedArray(self, length: int, updates: List[List[int]]) -> List[int]:
        tree = BinaryIndexedTree(length)
        for start, end, inc in updates:#nlogn
            tree.update(start + 1, inc)
            tree.update(end + 2, -inc)
        return [tree.query(i + 1) for i in range(length)]#获得前缀数组
if __name__=="__main__":
    solution=Solution()
    print(solution.getModifiedArray(5,[[1,3,2]]))
    
    print(solution.getModifiedArray(5,[[1,3,2],[2,4,3],[0,2,-2]]))

[0, 2, 2, 2, 0]
[-2, 0, 3, 5, 3]


定义一个lowbit函数，返回参数转为二进制后,最后一个1的位置所代表的数值。

例如，lowbit(34)的返回值将是2；而lowbit(12)返回4；lowbit(8)返回8。

将34转为二进制为(0010 0010)2。这里的“最后一个1”指的是从{\displaystyle 2^{0}}2^{0}位往前数，见到的第一个1，也就是{\displaystyle 2^{1}}2^{1}位上的1。

程序上，(~i + 1) & i表明了最后一位1的值。

仍然以34为例，~(0010 0010)的结果是1101 1101（221），加1后为1101 1110（222），把0010 0010与1101 1110作AND，得0000 0010（2）。

lowbit的一个简便求法:（C++）